In [1]:
import sys
sys.path.append("..")
from webapp.utils.azure_utils import KeyVault, DataLake
from webapp.utils.data_pre_utils import df_val_map
import numpy as np
import pandas as pd
import fastparquet
import os

In [2]:
# Connect to Storage Account
vault = KeyVault(keyVaultName = "keyvaultdva2022")
storage_credential = vault.get_secret(secretName = "storagePrimaryKey")
storage = DataLake(account_name = "storageaccountdva", credential = storage_credential)

In [3]:
file_system = "energyhub"
directory_raw = "/data_original/meters/cleaned"
directory_clean = "/data_parq/meters"

In [4]:
paths = storage.list_directory_contents(file_system, directory_raw)
file_names = []
for file in paths:
    file_names.append(file.split("/")[3])

In [5]:
file_names

['chilledwater_cleaned.csv',
 'electricity_cleaned.csv',
 'gas_cleaned.csv',
 'hotwater_cleaned.csv',
 'irrigation_cleaned.csv',
 'solar_cleaned.csv',
 'steam_cleaned.csv',
 'water_cleaned.csv']

In [6]:
for name in file_names:

    new_name = name.split("_")[0]+".parq"
    col_name = name.split("_")[0]
    df = storage.pandas_read(file_system, directory_raw, file_name=name)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.iloc[:, 1:] = df.iloc[:, 1:].astype("float64")
    df_tf = df.melt(id_vars='timestamp', var_name= 'building_id', value_name= col_name)
    df_tf = df_tf.astype({'building_id': 'category'})
    #df_tf = df_tf.set_index("timestamp")
    assert len(df) *(len(df.columns)-1) == len(df_tf)

    df_tf.to_parquet(path = new_name, engine = "pyarrow", compression = "gzip", index = False)
    storage.upload(file_system, directory = directory_clean, file_name = new_name, file_path = new_name, overwrite=True)
    os.remove(new_name)   

chilledwater.parq write complete
electricity.parq write complete
gas.parq write complete
hotwater.parq write complete
irrigation.parq write complete
solar.parq write complete
steam.parq write complete
water.parq write complete
